In [5]:
#既に作ったベクトルを効率的に処理するためのプログラム。改変前にロードしておく
import os,pickle
vecotrs_list = os.listdir("./embedding_vectors")
print(vecotrs_list)
prev_full_vectors_dict = {}
for vector in vecotrs_list:
    with open(f"./embedding_vectors/{vector}","rb") as f:
        vectors_dict = pickle.load(f)
        prev_full_vectors_dict.update(vectors_dict)

for title in prev_full_vectors_dict:
    print(title)

['embedding_vectors1000.pickle', 'embedding_vectors10000.pickle', 'embedding_vectors11000.pickle', 'embedding_vectors12000.pickle', 'embedding_vectors13000.pickle', 'embedding_vectors14000.pickle', 'embedding_vectors15000.pickle', 'embedding_vectors16000.pickle', 'embedding_vectors17000.pickle', 'embedding_vectors18000.pickle', 'embedding_vectors19000.pickle', 'embedding_vectors2000.pickle', 'embedding_vectors20000.pickle', 'embedding_vectors20132.pickle', 'embedding_vectors3000.pickle', 'embedding_vectors4000.pickle', 'embedding_vectors5000.pickle', 'embedding_vectors6000.pickle', 'embedding_vectors7000.pickle', 'embedding_vectors8000.pickle', 'embedding_vectors9000.pickle']
クックとプッケ
スピード太郎
タンタンのコンゴ探険
ブロンディ (漫画)
タンタン アメリカへ
のらくろ
ファラオの葉巻
ファニーズ・オンパレード
冒険ダン吉
青い蓮
タンクタンクロー
フラッシュ・ゴードン
かけた耳
リトル・ルル
ザ・ファントム
フクちゃん
黒い島のひみつ
アクション・コミックス
オトカル王の杖
スーパーマン
金のはさみのカニ
グリーンランタン
ジャスティス・ソサエティ・オブ・アメリカ
仲よし手帖
バットマン (コミック)
フラッシュ (DCコミックス)
アクアマン
グリーンアロー
ふしぎな流れ星
プラスチックマン
翼賛一家大和さん
ワンダーウーマン
なぞのユニコーン号
サザエさん
マァチャンの日記帳
ヤ

In [11]:
for title,data in prev_full_vectors_dict.items():
    print(len(data))
    asdf

3


NameError: name 'asdf' is not defined

In [13]:
#検索用の行列を作成します,まずは行列を作る前にtextembeddingで取得したベクトルデータを小分けにして保存する
from create_embedding_vector_from_res import create_embedding_vector_from_res,encode_filename
import bs4
from bs4 import BeautifulSoup
import requests
import pickle,re,os
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

count = 0 #これが一定以上になったらpickleとしてセーブする
page_title_and_embedding_vectors_dict = {}
for page_title,data in info_for_search_dict.items():
    print(f"{count}:{page_title}")
    if page_title in prev_full_vectors_dict:
        page_title_and_embedding_vectors_dict[page_title] = prev_full_vectors_dict[page_title] #以前のものが使いまわしできる場合は使う！
        # print(len(page_title_and_embedding_vectors_dict[page_title]))
    else:
        year = data["page_year"]
        with open(f"../../data_for_search/main_wiki_html/{year}/{encode_filename(page_title)}.pickle","rb") as f:
            res = pickle.load(f)
        soup = BeautifulSoup(res, 'html.parser')
        
        result =  create_embedding_vector_from_res(soup)
        page_title_and_embedding_vectors_dict[page_title] = {"overview_embedding":result[0],"story_embedding":result[1],"character_embedding":result[2]}
    count+=1
    if count %1000==0:
        with open(f"./embedding_vectors/embedding_vectors{count}.pickle","wb") as f:
            pickle.dump(page_title_and_embedding_vectors_dict,f)
            page_title_and_embedding_vectors_dict = {}
#最後に余剰分を保存する
with open(f"./embedding_vectors/embedding_vectors{count}.pickle","wb") as f:
    pickle.dump(page_title_and_embedding_vectors_dict,f)
    page_title_and_embedding_vectors_dict = {}

0:クックとプッケ
1:スピード太郎
2:タンタンのコンゴ探険
3:ブロンディ (漫画)
4:タンタン アメリカへ
5:のらくろ
6:ファラオの葉巻
7:ファニーズ・オンパレード
8:冒険ダン吉
9:青い蓮
10:タンクタンクロー
11:フラッシュ・ゴードン
12:かけた耳
13:リトル・ルル
14:ザ・ファントム
15:フクちゃん
16:黒い島のひみつ
17:アクション・コミックス
18:オトカル王の杖
19:ジャングルの女王シーナ
20:スーパーマン
21:金のはさみのカニ
22:グリーンランタン
23:ジャスティス・ソサエティ・オブ・アメリカ
24:仲よし手帖
25:バットマン (コミック)
26:フラッシュ (DCコミックス)
27:アクアマン
28:グリーンアロー
29:ふしぎな流れ星
30:プラスチックマン
31:翼賛一家大和さん
32:ワンダーウーマン
33:なぞのユニコーン号
34:サザエさん
35:マァチャンの日記帳
36:ヤネウラ3ちゃん
37:ロストワールド (漫画)
38:火星博士
39:新宝島
40:バット君
41:ムーミンまんがシリーズ
42:リル・フォークス
43:吸血魔団
44:大空魔王
45:地底国の怪人
46:ペ子ちゃん
47:妖怪探偵團
48:リツル・ペドロ
49:あんみつ姫
50:沙漠の魔王
51:スーパーボーイ
52:デンスケ (漫画)
53:似たもの一家
54:メトロポリス (漫画)
55:黄金都市
56:ジャングル大帝
57:ピーナッツ (漫画)
58:プーサン
59:ふしぎ旅行記
60:化石島
61:来るべき世界 (漫画)
62:クリちゃん
63:グンナイ君
64:新世界ルルー
65:新やじきた道中記
66:天使の玉ちゃん
67:ピピちゃん
68:わんぱくデニス
69:太平洋Xポイント
70:鉄腕アトム
71:ぼくのそんごくう
72:町子かぶき迷作集
73:リボンの騎士
74:リンゴ園の少女
75:ロック冒険記
76:スーパーデューパーマン
77:トンちゃん
78:UTOPIA 最後の世界大戦
79:赤胴鈴之助
80:地球の悪魔
81:犯人は誰だ!!
82:火の鳥 (漫画)
83:まっぴら君
84:空気男爵
85:白面夜叉
86:ぼんこちゃん
87:ロボット三等兵
88:アッちゃん
89:嵐をこえて
90:仙人部落
9

In [21]:
#前のコードでpage_title_and_embedding_vectors_dictを保存後に初期化するのを忘れていたため、数が増加するごとにめちゃくちゃ大きくなってしまっていた・・・これはよくない！
# 解決したのでコメントアウトしました
# import os 
# embedding_vectors_pickle_list = os.listdir("./embedding_vectors/")
# embedding_vectors_pickle_dict = {int(embedding_vectors_pickle.replace(".pickle","").replace("embedding_vectors","")):embedding_vectors_pickle for embedding_vectors_pickle in embedding_vectors_pickle_list}
# embedding_vectors_pickle_dict = sorted(embedding_vectors_pickle_dict.items())
# embedding_vectors_pickle_list = list(y for x,y in embedding_vectors_pickle_dict) #必要なのはpickleの名前だけなのでそれを取り出す

# full_page_title_list = []
# for index, embedding_vectors_pickle  in enumerate(embedding_vectors_pickle_list):
#     with open(f"./embedding_vectors/{embedding_vectors_pickle}","rb") as f:
#         data = pickle.load(f)
    
#     for page_title in full_page_title_list:
#         if page_title in data:data.pop(page_title)
    
#     for page_title in data:
#         full_page_title_list.append(page_title)
    
#     # for page_title in data:
#     #     print(page_title)
#     with open(f"./embedding_vectors/{embedding_vectors_pickle}","wb") as f:
#         pickle.dump(data,f)


In [14]:
#保存したembedding_vectorsを元にmatrixを作ります(ベクトルから漫画の類似度を出力できるかテスト.ipynb　から拝借)

import os, json, pickle,re

# import numpy as torch
import torch
title_list = []
overview_embedding_matrix = torch.empty((0,512))
story_embedding_matrix = torch.empty((0,512))
character_embedding_matrix = torch.empty((0,512))
for embedding_pickle in os.listdir("./embedding_vectors/"):

    # if "2016" not in embedding_json: continue #2023年のデータだけとってくる。開発用・完成したらここはコメントアウト！

    with open("./embedding_vectors/" + embedding_pickle ,"rb") as f:
        pickle_file = pickle.load(f)
        number = int(embedding_pickle.replace(".pickle","").replace("embedding_vectors",""))
        print(f"{number}のベクトルデータを処理中...")
        for page_title,data in pickle_file.items():
            title = page_title
            title_list.append(title)
            overview_embedding = torch.tensor(data["overview_embedding"])
            story_embedding = torch.tensor(data["story_embedding"])
            character_embedding = torch.tensor(data["character_embedding"])

            overview_embedding_matrix = torch.vstack((overview_embedding_matrix,overview_embedding))
            story_embedding_matrix = torch.vstack((story_embedding_matrix,story_embedding))
            character_embedding_matrix = torch.vstack((character_embedding_matrix,character_embedding))
            
            
print(overview_embedding_matrix.shape)
print(story_embedding_matrix.shape)
print(character_embedding_matrix.shape)

embedding_matrix_list =[overview_embedding_matrix,story_embedding_matrix,character_embedding_matrix ]

c:\Users\tanigutirei\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1000のベクトルデータを処理中...
10000のベクトルデータを処理中...
11000のベクトルデータを処理中...
12000のベクトルデータを処理中...
13000のベクトルデータを処理中...
14000のベクトルデータを処理中...
15000のベクトルデータを処理中...
16000のベクトルデータを処理中...
17000のベクトルデータを処理中...
18000のベクトルデータを処理中...
19000のベクトルデータを処理中...
2000のベクトルデータを処理中...
20000のベクトルデータを処理中...
20132のベクトルデータを処理中...
20233のベクトルデータを処理中...
3000のベクトルデータを処理中...
4000のベクトルデータを処理中...
5000のベクトルデータを処理中...
6000のベクトルデータを処理中...
7000のベクトルデータを処理中...
8000のベクトルデータを処理中...
9000のベクトルデータを処理中...
torch.Size([20365, 512])
torch.Size([20365, 512])
torch.Size([20365, 512])


In [ ]:
#matrixをhstackしてから保存しようとしたがfilesizeが100mbを超えたのでなくなく断念・・・そのため分割して保存します
import pickle
with open("../../data_for_search/title_and_embedding_matrix/title_list.pickle","wb") as f:
    pickle.dump(title_list,f)

with open("../../data_for_search/title_and_embedding_matrix/overview_embedding_matrix.pickle","wb") as f:
    pickle.dump(overview_embedding_matrix,f)

with open("../../data_for_search/title_and_embedding_matrix/story_embedding_matrix.pickle","wb") as f:
    pickle.dump(story_embedding_matrix,f)

with open("../../data_for_search/title_and_embedding_matrix/character_embedding_matrix .pickle","wb") as f:
    pickle.dump(character_embedding_matrix ,f)
